# 2nd try

## imports

In [1]:
import pandas as pd
import numpy as np
from sklearn import cross_validation
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline

## read csv

In [2]:
train = pd.read_csv('../../data/train.csv', parse_dates = ['Date'])
test = pd.read_csv('../../data/test.csv', parse_dates = ['Date'])
stores = pd.read_csv('../../data/store.csv')

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## helper

In [3]:
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w

def RMSPE(y, yhat):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe

In [4]:
rmspe_scorer = make_scorer(RMSPE, greater_is_better=False)
rfr = RandomForestRegressor(n_estimators=30)

## normalize & standardize

In [11]:
def normalize_data(sales, stores):
    sales['year'] = sales.Date.apply(lambda x: x.year)
    sales['month'] = sales.Date.apply(lambda x: x.month)
    sales['day'] = sales.Date.apply(lambda x: x.day)
    sales['woy'] = sales.Date.apply(lambda x: x.weekofyear)
    sales['StateHoliday'] = sales['StateHoliday'].apply(lambda x: 0 if '0' == x else 1)

    merged_data = sales.merge(stores, on='Store', copy=False)
    merged_data=pd.get_dummies(merged_data, columns=['StoreType', 'Assortment', 'DayOfWeek'])
    merged_data.drop(['PromoInterval', 'Date'], axis=1, inplace=True)
    merged_data = merged_data.fillna(0)
    
    return merged_data

## just do it

In [12]:
data = normalize_data(train, stores)
X_train = data.drop(['Sales', 'Customers'], axis=1).values
y_train = data['Sales'].values
fold = cross_validation.KFold(n=len(y_train), n_folds=4, shuffle=True, random_state=42)

In [13]:
# n_jobs number of cores
scores = cross_validation.cross_val_score(rfr, X_train, y_train, scoring=rmspe_scorer, cv=fold, n_jobs=4)
scores.mean()

-0.18083668883307236